In [1]:
"""
Code to process the XYZ file produced by open3dqsar into pandas
The ASCII did not have grid coords only the energy values
The param for the format keyword in export is 'XYZ'
Run this code and specify the file name in the fin line
Last few cells are the beginnings of a function to process the entire directory

###############################
import type=dependent file="C:\data\Datasets.Py-CoMFA\07.THERM\therm_binding_values.txt"

##########################
Below is the command for open3dqsar to generate the MIF for python
##########################

import type=mol2 file="C:\data\comb_therm_train.mol2"

box step=1.0

calc_field type=VDW

calc_field type=MM_ELE

export type=OBJECT_FIELD file=therm format=XYZ 

 


"""

'\nCode to process the XYZ file produced by open3dqsar into pandas\nThe ASCII did not have grid coords only the energy values\nThe param for the format keyword in export is \'XYZ\'\nRun this code and specify the file name in the fin line\nLast few cells are the beginnings of a function to process the entire directory\n\n###############################\nimport type=dependent file="C:\\data\\Datasets.Py-CoMFA\x07.THERM\therm_binding_values.txt"\n\n##########################\nBelow is the command for open3dqsar to generate the MIF for python\n##########################\n\nimport type=mol2 file="C:\\data\\comb_therm_train.mol2"\n\nbox step=1.0\n\ncalc_field type=VDW\n\ncalc_field type=MM_ELE\n\nexport type=OBJECT_FIELD file=therm format=XYZ \n\n \n\n\n'

In [2]:
import os
import sys
import pandas as pd
import numpy as  np
import time
import os
import shutil

In [3]:
def set_globals():
    global open3D_directory
    global dataset_name_num
    global MIF_nodes 
    global xyz_offset
    open3D_directory = "C:\open3dtools"
    dataset_name_num = "3u5l"
    MIF_nodes = [23,27,21]
    xyz_offset = [25,26,26]

In [4]:
def agrd_to_array(entry_path):
    fin = open(entry_path, "rt")
    fout = open("out.txt", "wt")

    for line in fin:
        fout.write(' '.join(line.split()))
        fout.write("\n")

    fin.close()
    fout.close()  
    npdata = np.loadtxt("out.txt",skiprows=5,delimiter=" ")
    MIF = np.zeros(MIF_nodes)
    for line in npdata:
        
        x,y,z,target = line
        x -= xyz_offset[0]
        y -= xyz_offset[1]
        z -= xyz_offset[2]
        MIF[int(x),int(y),int(z)] = target
    dataset_name = entry_path[15:].split("_")[0]
    number = entry_path[:-5].split("-")[-1]
    field = entry_path.split("_")[1]
    new_path =str(number+"_"+field+'.npy')    
    np.save(f"data/EllenMIF/{dataset_name_num}/{new_path}", MIF)

In [5]:
def process_dataset():
    itr = 0
    parent_dir = r"C:\Users\Linden\GitHub\3DQSAR\data\EllenMIF"
    new_dir_path = os.path.join(parent_dir, dataset_name_num)
    os.mkdir(new_dir_path)
    
    for entry in os.scandir(open3D_directory):
        if (entry.path.endswith(".agrd")):
            name = entry.path
            agrd_to_array(name)
        if itr%50==0:
            print(itr)
        itr+=1
            
            

In [6]:
def generate_labels():
    path = os.path.join(r"C:\Users\Linden\GitHub\3DQSAR\data\DatasetsEdited",dataset_name_num)
    tr = pd.read_csv(os.path.join(path,"pIC50.tr.txt"),
                     header=None,
                     delimiter=" ",
                     names=["Name","Y"],
                     index_col=False)
    
    if os.path.isfile(os.path.join(path,"pIC50.ts.txt")) == True:
        ts = pd.read_csv(os.path.join(path,"pIC50.ts.txt"),
                         header=None,
                         delimiter=" ",
                         names=["Name","Y"],
                         index_col=False)
        
        Y = pd.concat([tr,ts],ignore_index=True)
    else:
        Y = tr
    Y.sort_values(by=['Name'],inplace=True,ignore_index=True)
    count = len(Y)
    Y['ID']=np.arange(1,count+1)
    Y.to_csv(path_or_buf = os.path.join("data/LabelledData/",
                                        dataset_name_num,
                                        "pIC50.csv"),
             index=False)

    
    

In [7]:
if __name__=="__main__":
    set_globals()
    process_dataset()
#     generate_labels()

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000


In [8]:
print('hi')

hi


In [9]:
"""

import type=mol2 file="C:\Users\Linden\GitHub\3DQSAR\data\DatasetsEdited\06.DHFR\Data\combined_AT2.sdf"

box step=1.0

pr



X start, X end coordinates:    14.0000, 59.0000
Y start, Y end coordinates:    1.0000, 77.0000
Z start, Z end coordinates:    -2.0000, 51.0000
X nodes:                       46
Y nodes:                       77
Z nodes:                       54

"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 27-28: truncated \UXXXXXXXX escape (<ipython-input-9-d6ac462014cb>, line 18)